Let's train a pre-trained model to recognize new image classes

### Let's Configure 

In [0]:
import numpy
import pandas as pd
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing import image
from pathlib import Path
from keras.applications import resnet50

Configure Kaggle in the runtime and download data from kaggle

In [0]:
#!mkdir ~/.kaggle
#!mv kaggle.json ~/.kaggle/
!kaggle competitions download -c dog-breed-identification

Unzip everything and remove zip files

In [0]:
!unzip -q test.zip
!unzip -q train.zip
!unzip -q labels.csv.zip

!rm -rf *.zip

In [0]:
!echo -e "affenpinscher\nafghan_hound\nafrican_hunting_dog\nairedale\namerican_staffordshire_terrier" > label_class.txt

Configure our paths

In [0]:
#Image paths for training and testing
train_path = Path("train")
test_path = Path("test")

#File paths for all info and the labels
labels_file_path = "labels.csv"
label_class_path = "label_class.txt"

### Let's Load our Data

In [0]:
data = pd.read_csv(labels_file_path)

# Read dog breeds into a dictionary
breeds = pd.read_csv(label_class_path, header=None).values
breeds_dic = {}
for i in range(len(breeds)):
    breeds_dic[breeds[i][0]] = i

image_names_col = pd.DataFrame(data, columns=['id']).values
labels_col = pd.DataFrame(data, columns=['breed']).values
data_dic = {}
for i in range(len(data)):
    data_dic[image_names_col[i][0]] = labels_col[i][0]

We read our images, resize them to 300x300 and store them inside the images list

In [0]:
images = []
labels = []


def load_convert_image(img_path):
    img = image.load_img(img_path, target_size=(300, 300))
    img_array = image.img_to_array(img)
    return img_array


for im in train_path.glob("*.jpg"):
    label = data_dic.get(im.stem)
    label_class = breeds_dic.get(label)
    if label_class is not None:
        labels.append(label_class)
        images.append(load_convert_image(im))

In [0]:
#Convert our labels into one hot encoding
#we use this as the output for our new CNN
y = to_categorical(labels)
print(str(labels[0]) + " " + str(labels[1]))
print(y[0])
print(y[1])

### Let's Configure the Model

In [0]:
#Download the ResNet50 model
#include top false means we remove the final dense layer
feature_model = resnet50.ResNet50(weights="imagenet",include_top=False, input_shape=(300, 300, 3))

In [0]:
#Preprocess the data
X = resnet50.preprocess_input(numpy.array(images))

#This is the output of the convo layers
#These will be the input for the new Dense layer
X_for_dense_model = feature_model.predict(X)

In [0]:
#Creating our new dense layer using keras Sequential()
model = Sequential()
model.add(Flatten(input_shape=X_for_dense_model.shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(breeds_dic), activation="softmax"))

###Train the model

In [0]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model.fit(
    X_for_dense_model,
    y,
    epochs=20,
    shuffle=True
)

###Test the model

In [0]:
#Let's feed in this image to the model
im = load_convert_image("test/00b29dbc49177cd4faeac3c485330af9.jpg")
ims = numpy.expand_dims(im, axis=0)

In [0]:
#Preprocess
x_test = resnet50.preprocess_input(numpy.array(ims))

#Get the output from the convo layers
x_test_for_dense_model = feature_model.predict(x_test)

#Feed the convo output as the input for prediction
result = model.predict(x_test_for_dense_model)

In [46]:
#Reverse the onehot encoding and display the result
from numpy import argmax
res = argmax(result)
print(breeds[4][0])

american_staffordshire_terrier


###Convert the model to a tf.lite

In [0]:
model.save('my_model.h5')

In [0]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model_file("my_model.h5")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)